In [39]:
import csv

from IPython.display import clear_output

import numpy as np
import pandas as pd
import plotly
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default='notebook'
plotly.io.orca.config.executable = "~/miniconda3/bin/orca"

In [2]:
basics_file = pd.read_table('title.basics.tsv',
    sep='\t',
    lineterminator='\n',
    quotechar='', # else rows with " in title wont work
    quoting=csv.QUOTE_NONE,
    dtype={
        'tconst': 'string',
        'titleType': 'string',
        'primaryTitle': 'string',
        'originalTitle': 'string',
        'isAdult': 'float',
        'startYear': 'float',
        'endYear': 'float',
        'runtimeMinutes': 'float',
        'genres': 'string'
        },
    na_values=['\\N']
)

In [3]:
# improvements
basics_file["startYear"] = basics_file["startYear"].replace(np.NaN, 2022.0)
basics_file["endYear"] = basics_file["endYear"].replace(np.NaN, 2022.0)
basics_file["startYear"] = pd.to_datetime(basics_file.startYear, format='%Y')
basics_file["endYear"] = pd.to_datetime(basics_file.endYear, format='%Y')
temp = basics_file["endYear"] + np.timedelta64(364, 'D')
basics_file["endYear"] = temp
basics_file["title_and_tconst"] = basics_file["primaryTitle"] + " " + basics_file["tconst"]

In [4]:
basics_file.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,title_and_tconst
0,tt0000001,short,Carmencita,Carmencita,0.0,1894-01-01,2022-12-31,1.0,"Documentary,Short",Carmencita tt0000001
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0.0,1892-01-01,2022-12-31,5.0,"Animation,Short",Le clown et ses chiens tt0000002
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0.0,1892-01-01,2022-12-31,4.0,"Animation,Comedy,Romance",Pauvre Pierrot tt0000003
3,tt0000004,short,Un bon bock,Un bon bock,0.0,1892-01-01,2022-12-31,12.0,"Animation,Short",Un bon bock tt0000004
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0.0,1893-01-01,2022-12-31,1.0,"Comedy,Short",Blacksmith Scene tt0000005


In [6]:
ratings_file = pd.read_table('title.ratings.tsv',
    sep='\t',
    lineterminator='\n',
    quotechar='', # else rows with " in title wont work
    quoting=csv.QUOTE_NONE,
    dtype={
        'tconst': 'string',
        'averageRating': 'float',
        'numVotes': 'int'
        },
    na_values=['\\N']
)

In [7]:
ratings_file.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1702
1,tt0000002,6.1,210
2,tt0000003,6.5,1462
3,tt0000004,6.2,123
4,tt0000005,6.2,2262


In [8]:
episode_file = pd.read_table('title.episode.tsv',
    sep='\t',
    lineterminator='\n',
    quotechar='', # else rows with " in title wont work
    quoting=csv.QUOTE_NONE,
    dtype={
        'tconst': 'string',
        'parentTconst': 'string',
        'seasonNumber': 'float',
        'episodeNumber': 'float'
        },
    na_values=['\\N']
)

In [9]:
episode_file.head()

,tconst,parentTconst,seasonNumber,episodeNumber
0,tt0041951,tt0041038,1.0,9.0
1,tt0042816,tt0989125,1.0,17.0
2,tt0042889,tt0989125,NaN,NaN
3,tt0043426,tt0040051,3.0,42.0
4,tt0043631,tt0989125,2.0,16.0


In [10]:
print("Amount of imdb movies/series/episodes/titles:", basics_file.shape)
print("Amount of ratings for movies/series/episodes/titles:", ratings_file.shape)
print("Amount of episodes connected to series:", episode_file.shape)

Amount of imdb movies/series/episodes/titles: (7933717, 10)
Amount of ratings for movies/series/episodes/titles: (1155733, 3)
Amount of episodes connected to series: (5790039, 4)


In [11]:
ratings_merged = pd.merge(basics_file, ratings_file, how="left", on="tconst")

In [12]:
ratings_merged.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,title_and_tconst,averageRating,numVotes
0,tt0000001,short,Carmencita,Carmencita,0.0,1894-01-01,2022-12-31,1.0,"Documentary,Short",Carmencita tt0000001,5.7,1702.0
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0.0,1892-01-01,2022-12-31,5.0,"Animation,Short",Le clown et ses chiens tt0000002,6.1,210.0
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0.0,1892-01-01,2022-12-31,4.0,"Animation,Comedy,Romance",Pauvre Pierrot tt0000003,6.5,1462.0
3,tt0000004,short,Un bon bock,Un bon bock,0.0,1892-01-01,2022-12-31,12.0,"Animation,Short",Un bon bock tt0000004,6.2,123.0
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0.0,1893-01-01,2022-12-31,1.0,"Comedy,Short",Blacksmith Scene tt0000005,6.2,2262.0


In [15]:
# fixes ratings
ratings_merged["averageRating"] = ratings_merged["averageRating"].replace(np.NaN, 0.0)

In [16]:
# filter parent titles
tv_series = ratings_merged[(ratings_merged["titleType"] == 'tvSeries') | (ratings_merged["titleType"] == 'tvMiniSeries')]

In [17]:
# filter recent populair
tv_popular = tv_series[(tv_series["numVotes"] > 20000) & (tv_series["averageRating"] > 7.0) & (tv_series["startYear"] > np.datetime64('2005-01-01'))]

In [18]:
print(tv_popular.shape)

(537, 12)


In [19]:
tv_popular.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,title_and_tconst,averageRating,numVotes
359303,tt0374463,tvMiniSeries,The Pacific,The Pacific,0.0,2010-01-01,2010-12-31,530.0,"Action,Adventure,Drama",The Pacific tt0374463,8.3,101949.0
381321,tt0397442,tvSeries,Gossip Girl,Gossip Girl,0.0,2007-01-01,2012-12-30,42.0,"Drama,Romance",Gossip Girl tt0397442,7.4,157077.0
404313,tt0421030,tvSeries,Big Love,Big Love,0.0,2006-01-01,2011-12-31,60.0,Drama,Big Love tt0421030,7.6,20252.0
440608,tt0458290,tvSeries,Star Wars: The Clone Wars,Star Wars: The Clone Wars,0.0,2008-01-01,2020-12-30,23.0,"Action,Adventure,Animation",Star Wars: The Clone Wars tt0458290,8.3,68639.0
442977,tt0460690,tvSeries,The Unit,The Unit,0.0,2006-01-01,2009-12-31,60.0,"Action,Drama,Thriller",The Unit tt0460690,8.1,20493.0


In [20]:
tv_popular_sorted = tv_popular.sort_values(by=["startYear", "endYear"])

In [21]:
fig = px.timeline(tv_popular_sorted,
    x_start="startYear",
    x_end="endYear",
    y="title_and_tconst",
    color="averageRating",
    hover_name="primaryTitle",
    hover_data=["averageRating", "numVotes", "genres"],
    color_continuous_scale='Bluered'
)
fig.update_yaxes(autorange="reversed") # otherwise tasks are listed from the bottom up
fig.show()

In [22]:
plotly.offline.plot(fig, filename='popular.html', auto_open=False) 

'popular.html'

In [23]:
fig.write_image("popular.svg", width=1080.0, height=15000.0)

In [24]:
fig = go.Figure(data=[go.Table(
    header=dict(values=['primaryTitle', 'startYear', 'endYear', 'tconst'],
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[tv_popular_sorted.primaryTitle, tv_popular_sorted.startYear, tv_popular_sorted.endYear, tv_popular_sorted.tconst],
               fill_color='lavender',
               align='left'))
])
fig.show()

In [25]:
# add info to episode
episode_merged = pd.merge(episode_file, ratings_merged, how="left", on="tconst")
# TODO remove empty / non merged
print("Amount of episodes connected to series found:", episode_merged.shape)
# rename episode only columns
episode_merged.rename(columns = {'tconst':'episodeTconst'}, inplace = True)
episode_merged.rename(columns = {'averageRating':'episodeAverageRating'}, inplace = True)
episode_merged.rename(columns = {'numVotes':'episodeNumVotes'}, inplace = True)
episode_merged.rename(columns = {'titleType':'episodeTitleType'}, inplace = True)
episode_merged.rename(columns = {'primaryTitle':'episodePrimaryTitle'}, inplace = True)
episode_merged.rename(columns = {'originalTitle':'episodeOriginalTitle'}, inplace = True)
episode_merged.rename(columns = {'isAdult':'episodeIsAdult'}, inplace = True)
episode_merged.rename(columns = {'startYear':'episodeStartYear'}, inplace = True)
episode_merged.rename(columns = {'endYear':'episodeEndYear'}, inplace = True)
episode_merged.rename(columns = {'runtimeMinutes':'episodeRuntimeMinutes'}, inplace = True)
episode_merged.rename(columns = {'genres':'episodeGenres'}, inplace = True)
episode_merged.rename(columns = {'title_and_tconst':'episode_title_and_tconst'}, inplace = True)
# add parent info to episode
episode_merged = pd.merge(episode_merged, tv_popular_sorted, how="left", left_on='parentTconst', right_on='tconst')
print("Amount of episodes merged to popular tv serie parent:", episode_merged.shape)
# remove episodes without parent
episode_merged = episode_merged[(episode_merged["titleType"] == 'tvSeries') | (episode_merged["titleType"] == 'tvMiniSeries')]
print("Amount of episodes actually has  popular tv serie parent:", episode_merged.shape)
# sort episodes
episode_merged = episode_merged.sort_values(by=["parentTconst", "episodeStartYear", "seasonNumber", "episodeNumber"])
# reset index
episode_merged = episode_merged.reset_index()
episode_merged.pop("index")
episode_merged.head()

Amount of episodes connected to series found: (5790039, 15)
Amount of episodes merged to popular tv serie parent: (5790039, 27)
Amount of episodes actually has  popular tv serie parent: (28697, 27)


,episodeTconst,parentTconst,seasonNumber,episodeNumber,episodeTitleType,episodePrimaryTitle,episodeOriginalTitle,episodeIsAdult,episodeStartYear,episodeEndYear,...,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,title_and_tconst,averageRating,numVotes
0,tt1576600,tt0374463,1.0,1.0,tvEpisode,Guadalcanal/Leckie,Guadalcanal/Leckie,0.0,2010-01-01,2022-12-31,...,The Pacific,The Pacific,0.0,2010-01-01,2010-12-31,530.0,"Action,Adventure,Drama",The Pacific tt0374463,8.3,101949.0
1,tt1576598,tt0374463,1.0,2.0,tvEpisode,Basilone,Basilone,0.0,2010-01-01,2022-12-31,...,The Pacific,The Pacific,0.0,2010-01-01,2010-12-31,530.0,"Action,Adventure,Drama",The Pacific tt0374463,8.3,101949.0
2,tt1576603,tt0374463,1.0,3.0,tvEpisode,Melbourne,Melbourne,0.0,2010-01-01,2022-12-31,...,The Pacific,The Pacific,0.0,2010-01-01,2010-12-31,530.0,"Action,Adventure,Drama",The Pacific tt0374463,8.3,101949.0
3,tt1576599,tt0374463,1.0,4.0,tvEpisode,Gloucester/Pavuvu/Banika,Gloucester/Pavuvu/Banika,0.0,2010-01-01,2022-12-31,...,The Pacific,The Pacific,0.0,2010-01-01,2010-12-31,530.0,"Action,Adventure,Drama",The Pacific tt0374463,8.3,101949.0
4,tt1576607,tt0374463,1.0,5.0,tvEpisode,Peleliu Landing,Peleliu Landing,0.0,2010-01-01,2022-12-31,...,The Pacific,The Pacific,0.0,2010-01-01,2010-12-31,530.0,"Action,Adventure,Drama",The Pacific tt0374463,8.3,101949.0


In [27]:
print(episode_merged.iloc[0])

episodeTconst                                  tt1576600
parentTconst                                   tt0374463
seasonNumber                                         1.0
episodeNumber                                        1.0
episodeTitleType                               tvEpisode
episodePrimaryTitle                   Guadalcanal/Leckie
episodeOriginalTitle                  Guadalcanal/Leckie
episodeIsAdult                                       0.0
episodeStartYear                     2010-01-01 00:00:00
episodeEndYear                       2022-12-31 00:00:00
episodeRuntimeMinutes                               52.0
episodeGenres                     Action,Adventure,Drama
episode_title_and_tconst    Guadalcanal/Leckie tt1576600
episodeAverageRating                                 8.0
episodeNumVotes                                   2147.0
tconst                                         tt0374463
titleType                                   tvMiniSeries
primaryTitle                   

In [28]:
print("Amount of unique parents:", len(episode_merged["parentTconst"].unique()))

Amount of unique parents: 537


In [29]:
episodes_in_year = episode_merged.groupby(['parentTconst','episodeStartYear']).size().reset_index().rename(columns={0:'count'})
episodes_in_year.head()

,parentTconst,episodeStartYear,count
0,tt0374463,2010-01-01,10
1,tt0397442,2007-01-01,11
2,tt0397442,2008-01-01,20
3,tt0397442,2009-01-01,24
4,tt0397442,2010-01-01,21


In [30]:
# add columns begin and end day in year
episode_merged["episodes_start"] = np.datetime64('2000-01-01')
episode_merged["episodes_end"] = np.datetime64('2000-01-01')
# because of sort count episodes
last_start_number = 0.0
last_parentTconst = ""
last_episodeStartYear = ""
last_seasonNumber = 0.0
# loop
episodes_len = len(episode_merged)
for i in range(episodes_len):
    clear_output(wait=True)
    print(i+1, "/", episodes_len)
    row = episode_merged.loc[i]
    current_parentTconst = row["parentTconst"]
    current_episodeStartYear = row["episodeStartYear"]
    current_seasonNumber = row["seasonNumber"]
    current_episodeNumber = row["episodeNumber"]
    current_episodes_in_year = episodes_in_year[(episodes_in_year["parentTconst"] == current_parentTconst) & (episodes_in_year["episodeStartYear"] == current_episodeStartYear)]
    if len(current_episodes_in_year) != 1:
        print(current_episodes_in_year)
        print(row)
        break
    current_episodes_in_year = current_episodes_in_year["count"].iloc[0]
    distributed = 31536000.0/current_episodes_in_year
    if last_parentTconst != current_parentTconst:
        last_start_number = 0.0
        last_parentTconst = current_parentTconst
    if last_episodeStartYear != current_episodeStartYear:
        last_start_number = 0.0
        last_episodeStartYear = current_episodeStartYear
    episodes_start_sec = int(distributed*last_start_number)
    episodes_end_sec = int(distributed*(last_start_number+1.0))
    episode_merged.loc[i, "episodes_start"] = current_episodeStartYear + np.timedelta64(episodes_start_sec, 's')
    episode_merged.loc[i, "episodes_end"] = current_episodeStartYear + np.timedelta64(episodes_end_sec, 's')
    last_start_number += 1.0


28697 / 28697


In [32]:
episode_merged["episodeEndYear"] = episode_merged["episodeStartYear"] + np.timedelta64(1, 'Y')

In [33]:
episode_merged.head()

,episodeTconst,parentTconst,seasonNumber,episodeNumber,episodeTitleType,episodePrimaryTitle,episodeOriginalTitle,episodeIsAdult,episodeStartYear,episodeEndYear,...,isAdult,startYear,endYear,runtimeMinutes,genres,title_and_tconst,averageRating,numVotes,episodes_start,episodes_end
0,tt1576600,tt0374463,1.0,1.0,tvEpisode,Guadalcanal/Leckie,Guadalcanal/Leckie,0.0,2010-01-01,2011-01-01 05:49:12,...,0.0,2010-01-01,2010-12-31,530.0,"Action,Adventure,Drama",The Pacific tt0374463,8.3,101949.0,2010-01-01 00:00:00,2010-02-06 12:00:00
1,tt1576598,tt0374463,1.0,2.0,tvEpisode,Basilone,Basilone,0.0,2010-01-01,2011-01-01 05:49:12,...,0.0,2010-01-01,2010-12-31,530.0,"Action,Adventure,Drama",The Pacific tt0374463,8.3,101949.0,2010-02-06 12:00:00,2010-03-15 00:00:00
2,tt1576603,tt0374463,1.0,3.0,tvEpisode,Melbourne,Melbourne,0.0,2010-01-01,2011-01-01 05:49:12,...,0.0,2010-01-01,2010-12-31,530.0,"Action,Adventure,Drama",The Pacific tt0374463,8.3,101949.0,2010-03-15 00:00:00,2010-04-20 12:00:00
3,tt1576599,tt0374463,1.0,4.0,tvEpisode,Gloucester/Pavuvu/Banika,Gloucester/Pavuvu/Banika,0.0,2010-01-01,2011-01-01 05:49:12,...,0.0,2010-01-01,2010-12-31,530.0,"Action,Adventure,Drama",The Pacific tt0374463,8.3,101949.0,2010-04-20 12:00:00,2010-05-27 00:00:00
4,tt1576607,tt0374463,1.0,5.0,tvEpisode,Peleliu Landing,Peleliu Landing,0.0,2010-01-01,2011-01-01 05:49:12,...,0.0,2010-01-01,2010-12-31,530.0,"Action,Adventure,Drama",The Pacific tt0374463,8.3,101949.0,2010-05-27 00:00:00,2010-07-02 12:00:00


In [36]:
fig = px.timeline(episode_merged,
    x_start="episodes_start",
    x_end="episodes_end",
    y="title_and_tconst",
    color="episodeAverageRating",
    hover_name="primaryTitle",
    hover_data=["averageRating", "numVotes", "genres"],
    color_continuous_scale='viridis'
)
fig.update_yaxes(autorange="reversed") # otherwise tasks are listed from the bottom up
plotly.offline.plot(fig, filename='episodes.html', auto_open=False)

'episodes.html'

In [37]:
fig.write_image("episodes.svg", width=1900.0, height=14000.0)

In [38]:
fig.write_image("episodes.png", width=1900.0, height=14000.0)